In [ ]:
#this code has been run in Google Colab using GPU, all pieces of code are programmed accordingly
#this piece of code is to determine if a GPU is connected and report the properties
import tensorflow as tf
import os
tf.test.gpu_device_name()

!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

In [ ]:
#downloads necessary libraries, creates a subdirectory named "data" and downloads the given dataset to that directory
! pip install -q kaggle
from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

! kaggle datasets download -d mengcius/cinic10

! mkdir data
! unzip cinic10.zip -d data

In [ ]:
import numpy as np #to do matrix and array operations quickly

from tensorflow.keras.models import Sequential #to create a model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout

from sklearn import metrics # to give reports about recall, precision, recall and confusion matrix


def try_cnn_comb(train_generator, test_generator, valid_generator): #takes 3 generators that will be used in training, test and validation; sends parameters of 6 different combination of CNNs to a function that will train that model
    layers = [2,3]
    kernels = [3,5]
    dropout = [0.2,0.7]
    for x in layers: # for each layer
        for y in kernels: # for each kernel size
          if y == 3: # if kernel size is 3, then try for all dropout rates
            for z in dropout:
              run_cnn_comb(x, y, z, train_generator, test_generator, valid_generator)
          else: # if kernel size is 5, then try for only 0.2 dropout rate
            run_cnn_comb(x, y, 0.2, train_generator, test_generator, valid_generator)           
            
def run_cnn_comb(layer_count, kernel_size, dropout_rate, train_generator, test_generator, valid_generator): #creates a model with given parameters, and passes the model and the data to another function that will fit the model and print a detailed report of the outcome
    model = Sequential() #creates a sequential model
    model.add(Conv2D(32, kernel_initializer='GlorotNormal', kernel_size=kernel_size, activation='relu', input_shape=(224,224,3))) #adds a 2D convolution layer
    model.add(Dropout(dropout_rate)) #adds a dropout layer with the given dropout rate
    for i in range(layer_count-1): #adds these layers again and again untill it reaches the requested amount of layers
        model.add(Conv2D(32, kernel_initializer='GlorotNormal', kernel_size=kernel_size, activation='relu'))
        model.add(Dropout(dropout_rate))
    model.add(Flatten()) #flattens the data to pass it to the final dense layer
    model.add(Dense(10, kernel_initializer='GlorotNormal', activation = 'softmax')) # adds the final layer that will give an outcome
    
    model.summary() # prints model's specifications

    fit_report_models(train_generator, test_generator, valid_generator, model) # sends the data and the model to a function that will fit the model and report the outcome with confusion matrix

def fit_report_models(train_generator, test_generator, valid_generator, model): # takes generator datas and a model to fit the model and report an outcome with statistical success rates and a confusion matrix
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #compiles the given model with adam optimizer
    
    STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size # calculates the amount of steps in an epoch to completely traverse the data
    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
    
    model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN+1, validation_data=valid_generator, validation_steps=STEP_SIZE_VALID+1, epochs=5) # trains the model

    predictions = model.predict_generator(test_generator, steps=STEP_SIZE_TEST+1) # finds predictions with the test data

    predicted_classes = np.argmax(predictions, axis=1) # gets most likely class

    true_classes = test_generator.classes # list of true class labels
    class_labels = list(test_generator.class_indices.keys()) # gets all the labels

    report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels) # print a table of recall, precision and f1 score for each label
    print(report)
    confusion_matrix = metrics.confusion_matrix(y_true=true_classes, y_pred=predicted_classes) # creates a confusion matrix
    print(confusion_matrix)
    

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator # since getting all the data at once will create memory issues, generator will be used

train_gen = ImageDataGenerator(rescale=1./255) # creates a generator object that will normalize pixel values between 0 and 1
train_generator = train_gen.flow_from_directory(
    directory="data/train/", #data will be taken from this path
    target_size=(224, 224), #data will be resized to 224x224
    color_mode="rgb", #pixel values will be rgb
    batch_size=64, # 64 datas will be taken in each step in each epoch
    class_mode="categorical", # data is categorical
    shuffle=True, # shuffle to data to better train the model
    seed=42 # use a fixed seed to get the same randomization each time to compare the success rates
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory="data/test/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    directory="data/valid/",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

In [ ]:
try_cnn_comb(train_generator, test_generator, val_generator) # with this generators, creates all 6 combinations of models in the project document and makes a report

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16 # imports VGG model to use the pretrained model
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model # to use a pretrained model as a "model" to add it to another model

modelvgg = VGG16(weights='imagenet') # loads vgg model

modelvgg.layers.pop() # pops last layer

for i in range(len(modelvgg.layers)): # makes each layer untrainable
  modelvgg.layers[i].trainable = False

model = Sequential() # creates a new model since we can't manipulate a pretrained model
model.add(Model(modelvgg.input, modelvgg.layers[-2].output)) # adds the pretrained model to this model without the last layer
model.add(Dense(1024, trainable = True, activation='relu')) # adds a 1024-node trainable dense layer 
model.add(Dense(10, trainable = True, activation = 'softmax')) # adds an output layer

model.summary() # prints a summary of the model

fit_report_models(train_generator, test_generator, val_generator, model) # fits the model and reports the outcome

for i in range(4):
  modelvgg.layers[(-1)*i].trainable = True # makes the last 4 layers of the pretrained vgg model trainable

model = Sequential() # creates a new model and follows the same steps with the previous model
model.add(Model(modelvgg.input, modelvgg.layers[-2].output))
model.add(Dense(1024, trainable = True, activation='relu'))
model.add(Dense(10, trainable = True, activation = 'softmax'))

model.summary()

fit_report_models(train_generator, test_generator, val_generator, model)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50 # imports resnet model to use the pretrained model

modelvgg = ResNet50(weights= None, input_shape= (224,224,3)) # follows the same steps with vgg model for the resnet model

modelvgg.layers.pop()

for i in range(len(modelvgg.layers)):
  modelvgg.layers[i].trainable = False


model = Sequential()
model.add(Model(modelvgg.input, modelvgg.layers[-2].output))
model.add(Dense(1024, trainable = True, activation='relu'))
model.add(Dense(10, trainable = True, activation = 'softmax'))

model.summary()

fit_report_models(train_generator, test_generator, val_generator, model)

for i in range(4):
  modelvgg.layers[(-1)*i].trainable = True

model = Sequential()
model.add(Model(modelvgg.input, modelvgg.layers[-2].output))
model.add(Dense(1024, trainable = True, activation='relu'))
model.add(Dense(10, trainable = True, activation = 'softmax'))

model.summary()

fit_report_models(train_generator, test_generator, val_generator, model)